# Effective ribo_compare
This is the code used for extracting the data from RDME-ODE simulation results and comparing various trajectories of species effective ribo with normal ribo numbers

In [1]:
%run env.ipynb
import pickle
import os
import numpy as np
from jLM.RDME import File as RDMEFile
import jLM
import json
import matplotlib.pyplot as plt
import seaborn as sns
from traj_analysis_rdme import *
from tqdm import tqdm
import pandas as pd
import logging

default, get data required

In [2]:
effective_ribo_traj_dir = "/data2/2024_Yeast_GS/my_current_code/rdme_ode_results/20250310_wtnoer_riboeff_60min"
normal_ribo_traj_dir = "/data2/2024_Yeast_GS/my_current_code/rdme_ode_results/20250310_wtnoer_60min"
fig_dir = os.path.join(effective_ribo_traj_dir, 'figures_effective_ribo_comparison/')

if not os.path.exists(fig_dir):
    os.makedirs(fig_dir)
# Configure logging
log_file = os.path.join(fig_dir, 'run_log.log')
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)


logging.info(f"This is the file to compare between all ribosomes and effective ribosomes data: {effective_ribo_traj_dir} and {normal_ribo_traj_dir}")



er_files = [f for f in os.listdir(effective_ribo_traj_dir) if f.startswith('yeast') and f.endswith('.lm')]
normal_ribo_files = [f for f in os.listdir(normal_ribo_traj_dir) if f.startswith('yeast') and f.endswith('.lm')]
traj_suff = "_ode.jsonl"

logging.info(f"effective ribo files: {er_files}")
logging.info(f"normal ribo files: {normal_ribo_files}")

# Initialize dictionaries to store data for each species
effective_ribo_species_data = {}
normal_ribo_species_data = {}
effective_ribo_ode_data = {}
normal_ribo_ode_data = {}
rdmeTs = None
odeTs = None

# Process effective ribo files
for traj_file in tqdm(er_files, desc="Processing effective ribo files", unit="file"):
    logging.info(f"Processing effective ribo file: {traj_file}")
    traj, odeTraj, region_traj = get_traj(effective_ribo_traj_dir, traj_file, traj_suff)
    curr_rdmeTs, rdmeYs, curr_odeTs, odeYs, _, _ = get_data_for_plot(traj, odeTraj, region_traj=None, sparse_factor=1)
    
    if rdmeTs is None:
        rdmeTs = curr_rdmeTs
        odeTs = curr_odeTs

    for species, data in rdmeYs.items():
        if species not in effective_ribo_species_data:
            effective_ribo_species_data[species] = []
        effective_ribo_species_data[species].append(data)

    for species, data in odeYs.items():
        if species not in effective_ribo_ode_data:
            effective_ribo_ode_data[species] = []
        effective_ribo_ode_data[species].append(data)

# Process normal ribo files
for traj_file in tqdm(normal_ribo_files, desc="Processing normal ribo files", unit="file"):
    logging.info(f"Processing normal ribo file: {traj_file}")
    traj, odeTraj, _ = get_traj(normal_ribo_traj_dir, traj_file, traj_suff)
    NAV = 6.022e23 * (traj.reg.cytoplasm.volume + traj.reg.nucleoplasm.volume + traj.reg.plasmaMembrane.volume)
    _, rdmeYs, _, odeYs, _, _ = get_data_for_plot(traj, odeTraj, region_traj=None, sparse_factor=1)

    for species, data in rdmeYs.items():
        if species not in normal_ribo_species_data:
            normal_ribo_species_data[species] = []
        normal_ribo_species_data[species].append(data)

    for species, data in odeYs.items():
        if species not in normal_ribo_ode_data:
            normal_ribo_ode_data[species] = []
        normal_ribo_ode_data[species].append(data)

# Calculate and save effective ribo statistics
effective_ribo_results = []
for species, trajectories in effective_ribo_species_data.items():
    trajectories_array = np.array(trajectories)
    avg = np.mean(trajectories_array, axis=0)
    std = np.std(trajectories_array, axis=0)
    
    effective_ribo_results.append({
        'Species': f"RDME_{species}",
        'Time': ','.join(map(str, rdmeTs)),
        'Average': ','.join(map(str, avg)),
        'Std': ','.join(map(str, std))
    })

for species, trajectories in effective_ribo_ode_data.items():
    trajectories_array = np.array(trajectories)
    avg = np.mean(trajectories_array, axis=0)
    std = np.std(trajectories_array, axis=0)
    
    effective_ribo_results.append({
        'Species': f"ODE_{species}",
        'Time': ','.join(map(str, odeTs)),
        'Average': ','.join(map(str, avg)),
        'Std': ','.join(map(str, std))
    })

# Calculate and save normal ribo statistics
normal_ribo_results = []
for species, trajectories in normal_ribo_species_data.items():
    trajectories_array = np.array(trajectories)
    avg = np.mean(trajectories_array, axis=0)
    std = np.std(trajectories_array, axis=0)
    
    normal_ribo_results.append({
        'Species': f"RDME_{species}",
        'Time': ','.join(map(str, rdmeTs)),
        'Average': ','.join(map(str, avg)),
        'Std': ','.join(map(str, std))
    })

for species, trajectories in normal_ribo_ode_data.items():
    trajectories_array = np.array(trajectories)
    avg = np.mean(trajectories_array, axis=0)
    std = np.std(trajectories_array, axis=0)
    
    normal_ribo_results.append({
        'Species': f"ODE_{species}",
        'Time': ','.join(map(str, odeTs)),
        'Average': ','.join(map(str, avg)),
        'Std': ','.join(map(str, std))
    })

# Save to CSV files
effective_ribo_df = pd.DataFrame(effective_ribo_results)
normal_ribo_df = pd.DataFrame(normal_ribo_results)

effective_ribo_csv_path = os.path.join(fig_dir, 'effective_ribo_species_statistics.csv')
normal_ribo_csv_path = os.path.join(fig_dir, 'normal_ribo_species_statistics.csv')

effective_ribo_df.to_csv(effective_ribo_csv_path, index=False)
normal_ribo_df.to_csv(normal_ribo_csv_path, index=False)

logging.info(f"effective ribo statistics saved to: {effective_ribo_csv_path}")
logging.info(f"normal ribo statistics saved to: {normal_ribo_csv_path}")


2025-03-26 18:58:01,044 - INFO - This is the file to compare between all ribosomes and effective ribosomes data: /data2/2024_Yeast_GS/my_current_code/rdme_ode_results/20250310_wtnoer_riboeff_60min and /data2/2024_Yeast_GS/my_current_code/rdme_ode_results/20250310_wtnoer_60min
2025-03-26 18:58:01,045 - INFO - effective ribo files: ['yeast1.14.3mt20250305_24_t60.0minGAE11.1mMnoERefribowt_gpu4.lm', 'yeast1.14.3mt20250305_23_t60.0minGAE11.1mMnoERefribowt_gpu4.lm', 'yeast1.14.3mt20250305_25_t60.0minGAE11.1mMnoERefribowt_gpu4.lm', 'yeast1.14.3mt20250305_22_t60.0minGAE11.1mMnoERefribowt_gpu4.lm', 'yeast1.14.3mt20250305_21_t60.0minGAE11.1mMnoERefribowt_gpu4.lm']
2025-03-26 18:58:01,046 - INFO - normal ribo files: ['yeast1.14.3mt20250304_4_t60.0minGAE11.1mMnoERwt_gpu4.lm', 'yeast1.14.3mt20250304_5_t60.0minGAE11.1mMnoERwt_gpu4.lm', 'yeast1.14.3mt20250304_3_t60.0minGAE11.1mMnoERwt_gpu4.lm', 'yeast1.14.3mt20250303_1_t60.0minGAE11.1mMnoERwt_gpu4.lm', 'yeast1.14.3mt20250303_2_t60.0minGAE11.1mMnoERwt

no region data


Processing effective ribo files:  40%|████      | 2/5 [00:03<00:05,  1.72s/file]2025-03-26 18:58:04,482 - INFO - Processing effective ribo file: yeast1.14.3mt20250305_25_t60.0minGAE11.1mMnoERefribowt_gpu4.lm


no region data


Processing effective ribo files:  60%|██████    | 3/5 [00:05<00:03,  1.73s/file]2025-03-26 18:58:06,222 - INFO - Processing effective ribo file: yeast1.14.3mt20250305_22_t60.0minGAE11.1mMnoERefribowt_gpu4.lm


...

Processing normal ribo files:  80%|████████  | 4/5 [00:06<00:01,  1.70s/file]2025-03-26 18:58:16,406 - INFO - Processing normal ribo file: yeast1.14.3mt20250303_2_t60.0minGAE11.1mMnoERwt_gpu4.lm


no region data


Processing normal ribo files: 100%|██████████| 5/5 [00:08<00:00,  1.70s/file]

no region data



2025-03-26 18:58:18,577 - INFO - effective ribo statistics saved to: /data2/2024_Yeast_GS/my_current_code/rdme_ode_results/20250310_wtnoer_riboeff_60min/figures_effective_ribo_comparison/effective_ribo_species_statistics.csv
2025-03-26 18:58:18,578 - INFO - normal ribo statistics saved to: /data2/2024_Yeast_GS/my_current_code/rdme_ode_results/20250310_wtnoer_riboeff_60min/figures_effective_ribo_comparison/normal_ribo_species_statistics.csv


plot comparison graphs

In [3]:
# Read the saved statistics
effective_ribo_df = pd.read_csv(os.path.join(fig_dir, 'effective_ribo_species_statistics.csv'))
normal_ribo_df = pd.read_csv(os.path.join(fig_dir, 'normal_ribo_species_statistics.csv'))

# Function to convert string of comma-separated values to numpy array
def str_to_array(s):
    return np.array([float(x) for x in s.split(',')])

# Debug: logging.info available species
logging.info(f"Available species in effective ribo:  {effective_ribo_df['Species'].tolist()}")
logging.info(f"Available species in normal ribo:  {normal_ribo_df['Species'].tolist()}")

# Get unique species names directly from the CSV
unique_species = set(effective_ribo_df['Species'].unique()) | set(normal_ribo_df['Species'].unique())
logging.info(f"\nUnique species:  {unique_species}")

# Plot settings
plt.style.use('default')
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['figure.dpi'] = 600
plt.rcParams['font.size'] = 18  # Increase base font size
plt.rcParams['axes.titlesize'] = 28  # Increase title font size
plt.rcParams['axes.labelsize'] = 18  # Increase axis label font size
plt.rcParams['xtick.labelsize'] = 18  # Increase tick label font size
plt.rcParams['ytick.labelsize'] = 18  # Increase tick label font size
plt.rcParams['legend.fontsize'] = 18  # Increase legend font size

# Create plots for each species
for species_name in unique_species:
    fig, ax = plt.subplots()
    
    # Safely get data
    effective_ribo_species_data = effective_ribo_df[effective_ribo_df['Species'] == species_name]
    normal_ribo_species_data = normal_ribo_df[normal_ribo_df['Species'] == species_name]
    
    if len(effective_ribo_species_data) == 0 or len(normal_ribo_species_data) == 0:
        logging.info(f"Skipping {species_name} - data not found")
        continue
        
    er_data = effective_ribo_species_data.iloc[0]
    normal_ribo_data = normal_ribo_species_data.iloc[0]
    
    time = str_to_array(er_data['Time'])
    er_avg = str_to_array(er_data['Average'])
    er_std = str_to_array(er_data['Std'])
    normal_ribo_avg = str_to_array(normal_ribo_data['Average'])
    normal_ribo_std = str_to_array(normal_ribo_data['Std'])
    # get display name
    display_name = species_name.split('_', 1)[1] if '_' in species_name else species_name
    # Replace any subsequent underscores with colons
    display_name = display_name.replace('_', ':')
    # Plot effective ribo
    ax.plot(time, er_avg, label=f'Effective ribosomes', linestyle='-')
    ax.fill_between(time, er_avg - er_std, er_avg + er_std, alpha=0.2)
    
    # Plot normal ribo
    ax.plot(time, normal_ribo_avg, label=f'All ribosomes', linestyle='--')
    ax.fill_between(time, normal_ribo_avg - normal_ribo_std, normal_ribo_avg + normal_ribo_std, alpha=0.2)
    
    # Customize plot
    ax.set_xlabel('Time (min)')
    if species_name.startswith('RDME_DG'):  # Change y-label for species starting with 'DG'
        ax.set_ylabel('Probability')
    else:
        ax.set_ylabel('Counts')
    # ax.set_title(f'{species_n ame} Comparison')
    ax.legend(framealpha=0.3, loc='upper right')
    ax.grid(False)
    
    # Save figure
    plt.tight_layout()
    fig_path = os.path.join(fig_dir, f'{species_name}_comparison.png')
    plt.savefig(fig_path, dpi=600, bbox_inches='tight')
    logging.info(f"Saved plot for {display_name}")
    plt.close()

logging.info(f"\nPlots saved in: {fig_dir}")
logging.getLogger().handlers[0].flush() 

2025-03-26 18:58:18,639 - INFO - Available species in effective ribo:  ['RDME_DGrep', 'RDME_DGrep_G4d', 'RDME_DGrep_G4d_G80d', 'RDME_Rrep', 'RDME_Grep', 'RDME_DG1', 'RDME_DG1_G4d', 'RDME_DG1_G4d_G80d', 'RDME_R1', 'RDME_G1', 'RDME_DG2', 'RDME_DG2_G4d', 'RDME_DG2_G4d_G80d', 'RDME_R2', 'RDME_G2', 'RDME_DG3', 'RDME_DG3_G4d', 'RDME_DG3_G4d_G80d', 'RDME_R3', 'RDME_G3', 'RDME_G3i', 'RDME_DG4', 'RDME_R4', 'RDME_G4', 'RDME_G4d', 'RDME_DG80', 'RDME_DG80_G4d', 'RDME_DG80_G4d_G80d', 'RDME_R80', 'RDME_G80', 'RDME_G80d', 'RDME_G80d_G3i', 'RDME_ribosome', 'RDME_ribosomeR1', 'RDME_ribosomeR2', 'RDME_ribosomeR3', 'RDME_ribosomeR4', 'RDME_ribosomeR80', 'RDME_ribosomeGrep', 'ODE_GAI', 'ODE_G1', 'ODE_G1GAI', 'ODE_G2GAI', 'ODE_G2GAE', 'ODE_G2']
2025-03-26 18:58:18,640 - INFO - Available species in normal ribo:  ['RDME_DGrep', 'RDME_DGrep_G4d', 'RDME_DGrep_G4d_G80d', 'RDME_Rrep', 'RDME_Grep', 'RDME_DG1', 'RDME_DG1_G4d', 'RDME_DG1_G4d_G80d', 'RDME_R1', 'RDME_G1', 'RDME_DG2', 'RDME_DG2_G4d', 'RDME_DG2_G4d_G80

This is for G2 in the membrane

In [4]:
# Read the saved statistics
effective_ribo_df = pd.read_csv(os.path.join(fig_dir, 'effective_ribo_species_statistics.csv'))
normal_ribo_df = pd.read_csv(os.path.join(fig_dir, 'normal_ribo_species_statistics.csv'))

def str_to_array(s):
    return np.array([float(x) for x in s.split(',')])

# Calculate G2 totals for ER data
er_g2_data = effective_ribo_df[effective_ribo_df['Species'].isin(['ODE_G2', 'ODE_G2GAE', 'ODE_G2GAI'])].copy()
if len(er_g2_data) > 0:
    time = str_to_array(er_g2_data.iloc[0]['Time'])
    er_total = np.zeros_like(str_to_array(er_g2_data.iloc[0]['Average']))
    er_std_squared = np.zeros_like(er_total)
    
    for _, row in er_g2_data.iterrows():
        er_total += str_to_array(row['Average'])
        er_std_squared += str_to_array(row['Std'])**2
    er_total_std = np.sqrt(er_std_squared)

# Calculate G2 totals for NOER data
normal_ribo_g2_data = normal_ribo_df[normal_ribo_df['Species'].isin(['ODE_G2', 'ODE_G2GAE', 'ODE_G2GAI'])].copy()
if len(normal_ribo_g2_data) > 0:
    normal_ribo_total = np.zeros_like(str_to_array(normal_ribo_g2_data.iloc[0]['Average']))
    normal_ribo_std_squared = np.zeros_like(normal_ribo_total)
    
    for _, row in normal_ribo_g2_data.iterrows():
        normal_ribo_total += str_to_array(row['Average'])
        normal_ribo_std_squared += str_to_array(row['Std'])**2
    normal_ribo_total_std = np.sqrt(normal_ribo_std_squared)

# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(time, er_total, label='Effective ribosomes', linestyle='-')
plt.fill_between(time, er_total - er_total_std, er_total + er_total_std, alpha=0.2)

plt.plot(time, normal_ribo_total, label='All ribosomes', linestyle='--')
plt.fill_between(time, normal_ribo_total - normal_ribo_total_std, normal_ribo_total + normal_ribo_total_std, alpha=0.2)

plt.xlabel('Time (min)')
plt.ylabel('Counts')
# plt.title('Total G2 Comparison (G2 + G2GAE + G2GAI)')
plt.legend(framealpha=0.3, loc='upper right')
plt.grid(False)

# Save figure
plt.tight_layout()
fig_path = os.path.join(fig_dir, 'G2_membrane_comparison.png')
plt.savefig(fig_path, dpi=600, bbox_inches='tight')
logging.info(f"Saved plot for G2 total")
plt.close()
logging.getLogger().handlers[0].flush() 

2025-03-26 18:59:10,870 - INFO - Saved plot for G2 total


This is for GAI total

In [5]:
# Read the saved statistics
effective_ribo_df = pd.read_csv(os.path.join(fig_dir, 'effective_ribo_species_statistics.csv'))
normal_ribo_df = pd.read_csv(os.path.join(fig_dir, 'normal_ribo_species_statistics.csv'))

def str_to_array(s):
    return np.array([float(x) for x in s.split(',')])

# Create combined GAI species plot
fig, ax = plt.subplots(figsize=(10, 6))

# List of species to combine
gai_species = ['GAI', 'G1GAI', 'G3i', 'G2GAI']

# Initialize arrays for effective ribo and normal ribo data
effective_ribocombined_avg = None
effective_ribocombined_var = None
normal_ribo_combined_avg = None
normal_ribo_combined_var = None
time = None

# For tracking which species are actually used
effective_ribo_species_used = []
normal_ribo_species_used = []

# Combine effective ribo data
for species_name in gai_species:
    # Look for both ODE and RDME versions of the species
    matching_rows = effective_ribo_df[effective_ribo_df['Species'].str.contains(species_name)]
    
    if not matching_rows.empty:
        # Prefer ODE data if available
        effective_ribo_species_data = matching_rows[matching_rows['Species'].str.startswith('ODE')]
        if effective_ribo_species_data.empty:
            effective_ribo_species_data = matching_rows
            
        if len(effective_ribo_species_data) > 0:
            er_data = effective_ribo_species_data.iloc[0]
            # Track which species are being used
            effective_ribo_species_used.append(er_data['Species'])
            
            curr_avg = str_to_array(er_data['Average']) / NAV * 1e3
            curr_std = str_to_array(er_data['Std']) / NAV * 1e3
            curr_var = curr_std ** 2  # Convert std to variance
            
            if effective_ribocombined_avg is None:
                time = str_to_array(er_data['Time'])
                effective_ribocombined_avg = curr_avg
                effective_ribocombined_var = curr_var
            else:
                effective_ribocombined_avg += curr_avg
                effective_ribocombined_var += curr_var  # Variances add for independent variables

# Combine normal ribo data
for species_name in gai_species:
    # Look for both ODE and RDME versions of the species
    matching_rows = normal_ribo_df[normal_ribo_df['Species'].str.contains(species_name)]
    
    if not matching_rows.empty:
        # Prefer ODE data if available
        normal_ribo_species_data = matching_rows[matching_rows['Species'].str.startswith('ODE')]
        if normal_ribo_species_data.empty:
            normal_ribo_species_data = matching_rows
            
        if len(normal_ribo_species_data) > 0:
            normal_data = normal_ribo_species_data.iloc[0]
            # Track which species are being used
            normal_ribo_species_used.append(normal_data['Species'])
            
            curr_avg = str_to_array(normal_data['Average']) / NAV * 1e3
            curr_std = str_to_array(normal_data['Std']) / NAV * 1e3
            curr_var = curr_std ** 2  # Convert std to variance
            
            if normal_ribo_combined_avg is None:
                normal_ribo_combined_avg = curr_avg
                normal_ribo_combined_var = curr_var
            else:
                normal_ribo_combined_avg += curr_avg
                normal_ribo_combined_var += curr_var  # Variances add for independent variables

# Print which species were actually used
logging.info("Effective ribo species used in GAI total: " + str(effective_ribo_species_used))
logging.info("Normal ribo species used in GAI total: " + str(normal_ribo_species_used))

# Convert combined variances back to standard deviations
if effective_ribocombined_var is not None:
    effective_ribocombined_std = np.sqrt(effective_ribocombined_var)
if normal_ribo_combined_var is not None:
    normal_ribo_combined_std = np.sqrt(normal_ribo_combined_var)

# Plot effective ribo if data exists
if effective_ribocombined_avg is not None and time is not None:
    ax.plot(time, effective_ribocombined_avg, label='Effective ribosomes', linestyle='-')
    ax.fill_between(time, effective_ribocombined_avg - effective_ribocombined_std, 
                    effective_ribocombined_avg + effective_ribocombined_std, alpha=0.2)

# Plot normal ribo if data exists
if normal_ribo_combined_avg is not None:
    ax.plot(time, normal_ribo_combined_avg, label='All ribosomes', linestyle='--')
    ax.fill_between(time, normal_ribo_combined_avg - normal_ribo_combined_std, 
                    normal_ribo_combined_avg + normal_ribo_combined_std, alpha=0.2)
# Add horizontal line for GAE = 11.1mM with a more fitting color
ax.axhline(y=11.1, color='gray', linestyle='-.', linewidth=2, label='GAE')
ax.text(time[0]*1.05, 10.8, '11.1 mM', color='gray', fontsize=16, va='top', ha='left')

# Customize plot
ax.set_xlabel('Time (min)')
ax.set_ylabel('Counts')
# ax.set_title('Total GAI Species Comparison (GAI + G1GAI + G3i + G2GAI)')
ax.legend(framealpha=0.3, loc='upper right')
ax.grid(False)

# Save figure
plt.tight_layout()
fig_path = os.path.join(fig_dir, 'GAI_total_comparison.png')
plt.savefig(fig_path, dpi=600, bbox_inches='tight')
logging.info(f"Saved plot for GAI total")
plt.close()
logging.getLogger().handlers[0].flush()

2025-03-26 18:59:10,959 - INFO - Effective ribo species used in GAI total: ['ODE_GAI', 'ODE_G1GAI', 'RDME_G3i', 'ODE_G2GAI']
2025-03-26 18:59:10,960 - INFO - Normal ribo species used in GAI total: ['ODE_GAI', 'ODE_G1GAI', 'RDME_G3i', 'ODE_G2GAI']
2025-03-26 18:59:11,949 - INFO - Saved plot for GAI total
